In [1]:
import pandas as pd

df = pd.read_csv('listing.csv')
df2 = pd.read_csv('listing.csv')

print(df.columns)

Index(['ID', 'Category', 'Title', 'Description', 'Amenities', 'Location',
       'guests', 'beds', 'bathrooms', 'Price_Night', 'Cleanliness_rating',
       'Accuracy_rating', 'Communication_rating', 'Location_rating',
       'Check-in_rating', 'Value_rating', 'amenities_count', 'url', 'bedrooms',
       'Unnamed: 19'],
      dtype='object')


In [2]:
def remove_rows_with_missing_rating(df):
    if 'guests' in df.columns:
        df_cleaned = df[~df['guests'].astype(str).str.contains('Somerford Keynes England Unit')]
    else:
        df_cleaned = df.copy()
    
    numeric_columns = ['Cleanliness_rating', 'Accuracy_rating', 'Communication_rating', 'Location_rating', 'Check-in_rating', 'Value_rating']
    
    # Convert textual entries to 1 in numerical columns
    for column in numeric_columns:
        df_cleaned[column] = pd.to_numeric(df_cleaned[column], errors='coerce').fillna(1)
    
    # Remove rows with missing values
    df_cleaned = df_cleaned.dropna(subset=numeric_columns)
    
    return df_cleaned


def combine_description_strings(df):
    df['Description'] = df['Description'].str.replace(r'\s+', ' ')
    df['Description'] = df['Description'].str.replace(r'About this space', '')
    return df

def set_default_values(df):
    df['guests'] = df['guests'].fillna(1)
    df['beds'] =  df['beds'].fillna(1)
    df['bathrooms'] = df['bathrooms'].fillna(1)
    return df

In [10]:
import pandas as pd
from sklearn.impute import SimpleImputer

def load_airbnb(label='Category'):
    # Read the CSV file into a DataFrame
    df = pd.read_csv('listing.csv')
    
    # Filter out rows with missing 'Category' values
    df_cleaned = df.dropna(subset=[label])
    
    # Extract the label column
    labels = df_cleaned[label]
    
    # Remove the label column from the features
    features = df_cleaned.drop(columns=[label])
    
    return features, labels

def preprocess_data(features):
    # Impute missing values in numerical columns with the mean
    numerical_columns = ['Cleanliness_rating', 'Accuracy_rating', 'Communication_rating',
                         'Location_rating', 'Check-in_rating', 'Value_rating',
                         'guests', 'beds', 'bathrooms']
    non_numeric_columns = [col for col in features.columns if col not in numerical_columns]
    imputer = SimpleImputer(strategy='mean')
    features[numerical_columns] = imputer.fit_transform(features[numerical_columns])
    
    # Here, we'll drop non-numeric columns for simplicity
    features = features.drop(columns=non_numeric_columns)

    # Perform any other preprocessing steps as needed
    
    return features

# ... The rest of your preprocessing and modeling code ...

if __name__ == "__main__":
    # Load the dataset with 'Category' as the label
    features, labels = load_airbnb(label='Category')
    
    # Preprocess data
    features_preprocessed = preprocess_data(features)
    
    # The rest of your code for modeling and evaluation


ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'Somerford Keynes England United Kingdom'

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# from tabular_data import load_airbnb
from sklearn.impute import SimpleImputer

# Load the dataset with 'price_night' as the label
features, labels = load_airbnb(label='Category')

# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3, random_state=42)

# Impute missing values in features
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

# Split the data into training and testing sets
model = LogisticRegression(max_iter=1000,random_state=42)
model.fit(X_train_imputed,y_train)

# Make predictions on the test set
y_val_pred = model.predict(X_val_imputed)

# Calculate accuracy on the validation set
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", accuracy)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '2797a470-75c0-4ee3-9184-95f59c977701'

In [4]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tabular_data import load_airbnb

# Load Airbnb data
def load_airbnb(label='Category'):
    df = pd.read_csv('listing.csv')
    df_cleaned = df.dropna(subset=[label])
    df_cleaned = df_cleaned[~df['guests'].apply(lambda x: isinstance(x, str) and 'Somerford Keynes England Unit' in x)]
    labels = df_cleaned[label]
    features = df_cleaned.drop(columns=[label])
    return features, labels

# Preprocess data
def preprocess_data(features):
    numerical_columns = ['Cleanliness_rating', 'Accuracy_rating', 'Communication_rating',
                         'Location_rating', 'Check-in_rating', 'Value_rating',
                         'guests', 'beds', 'bathrooms']
    non_numeric_columns = [col for col in features.columns if col not in numerical_columns]

    imputer = SimpleImputer(strategy='mean')
    features[numerical_columns] = imputer.fit_transform(features[numerical_columns])

    features = features.drop(columns=non_numeric_columns)
    return features

# Load the dataset with 'Category' as the label
features, labels = load_airbnb(label='Category')

# Preprocess data
features_preprocessed = preprocess_data(features)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features_preprocessed, labels, test_size=0.3, random_state=42)

# Initialize and train the logistic regression model
logreg_model = LogisticRegression(max_iter=1000, random_state=42)
logreg_model.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = logreg_model.predict(X_val)

# Calculate accuracy on the validation set
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", accuracy)


Validation Accuracy: 0.3265993265993266


In [7]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load Airbnb data for classification
def load_airbnb_classification(label='Category'):
    df = pd.read_csv('listing.csv')
    df_cleaned = df.dropna(subset=[label])
    df_cleaned = df_cleaned[~df['guests'].apply(lambda x: isinstance(x, str) and 'Somerford Keynes England Unit' in x)]
    labels = df_cleaned[label]
    features = df_cleaned.drop(columns=[label])
    return features, labels

def preprocess_data(features):
    numerical_columns = ['Cleanliness_rating', 'Accuracy_rating', 'Communication_rating',
                         'Location_rating', 'Check-in_rating', 'Value_rating',
                         'guests', 'beds', 'bathrooms']
    non_numeric_columns = [col for col in features.columns if col not in numerical_columns]

    imputer = SimpleImputer(strategy='mean')
    features[numerical_columns] = imputer.fit_transform(features[numerical_columns])

    features = features.drop(columns=non_numeric_columns)
    return features

# Train a logistic regression model
def train_classification_model(X_train, y_train):
    logreg_model = LogisticRegression(max_iter=1000, random_state=42)
    logreg_model.fit(X_train, y_train)
    return logreg_model

def main():
    # Load the dataset with 'Category' as the label
    features, labels = load_airbnb_classification(label='Category')
    
    # Preprocess data
    features_preprocessed = preprocess_data(features)   
    
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(features_preprocessed, labels, test_size=0.3, random_state=42)
    
    
    # Train a classification model
    classification_model = train_classification_model(X_train, y_train)
    
    # Make predictions on training and test sets
    y_train_pred = classification_model.predict(X_train)
    y_test_pred = classification_model.predict(X_test)
    
    # Compute performance measures for training set
    accuracy_train = accuracy_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred, average='weighted')
    recall_train = recall_score(y_train, y_train_pred, average='weighted')
    f1_train = f1_score(y_train, y_train_pred, average='weighted')
    
    # Compute performance measures for test set
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred, average='weighted')
    recall_test = recall_score(y_test, y_test_pred, average='weighted')
    f1_test = f1_score(y_test, y_test_pred, average='weighted')
    
    # Print performance measures
    print("Training Accuracy:", accuracy_train)
    print("Training Precision:", precision_train)
    print("Training Recall:", recall_train)
    print("Training F1 Score:", f1_train)
    print("Test Accuracy:", accuracy_test)
    print("Test Precision:", precision_test)
    print("Test Recall:", recall_test)
    print("Test F1 Score:", f1_test)

if __name__ == "__main__":
    main()


Training Accuracy: 0.372463768115942
Training Precision: 0.3745577885274984
Training Recall: 0.372463768115942
Training F1 Score: 0.3495261716768263
Test Accuracy: 0.3265993265993266
Test Precision: 0.3402601880738268
Test Recall: 0.3265993265993266
Test F1 Score: 0.30944688387685926


In [6]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

def tune_classification_model_hyperparameters(model_class, param_grid, X_train, y_train, X_val, y_val):
    # Create a GridSearchCV object with accuracy scoring
    grid_search = GridSearchCV(model_class, param_grid, scoring='accuracy', cv=3)
    
    # Fit the grid search to the data
    grid_search.fit(X_train, y_train)
    
    # Get the best model and best hyperparameters
    best_model = grid_search.best_estimator_
    best_hyperparameters = grid_search.best_params_
    
    # Evaluate the best model on the validation set
    y_val_pred = best_model.predict(X_val)
    validation_accuracy = accuracy_score(y_val, y_val_pred)
    
    # Create a dictionary of performance metrics
    best_performance = {
        "validation_accuracy": validation_accuracy,
    }
    
    return best_model, best_hyperparameters, best_performance
